# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,4.59,77,26,14.73,TF,05/07/2024
1,1,st. john's,47.5649,-52.7093,9.60,89,100,3.09,CA,05/07/2024
2,2,sitka,57.0531,-135.3300,6.94,70,75,5.66,US,05/07/2024
3,3,preobrazheniye,42.9019,133.9064,3.69,90,99,1.85,RU,05/07/2024
4,4,rio grande,-32.0350,-52.0986,23.06,90,100,5.15,BR,05/07/2024


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "EsriImagery", 
    size = "Humidity", 
    scale = 0.7,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] < 29) & 
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Wind Speed'] < 4) & 
    (city_data_df['Cloudiness'] < 10) & 
    (city_data_df['Humidity'] < 80) & 
    (city_data_df['Humidity'] > 50)
]

# Drop any rows with null values
columns_to_check = ['Max Temp', 'Wind Speed', 'Cloudiness', 'Humidity']
ideal_cities_df = ideal_cities_df.dropna(subset=columns_to_check)

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
35,35,port alfred,-33.5906,26.8910,24.96,59,0,3.50,ZA,05/07/2024
107,107,brisas de zicatela,15.8369,-97.0419,27.29,74,0,1.03,MX,05/07/2024
171,171,santa terezinha,-29.9002,-50.0904,25.79,65,9,1.89,BR,05/07/2024
177,177,cidreira,-30.1811,-50.2056,26.08,63,9,1.36,BR,05/07/2024
236,236,teknaf,20.8624,92.3058,28.01,74,0,3.89,BD,05/07/2024
263,263,prado,-17.3411,-39.2208,28.61,67,8,3.79,BR,05/07/2024
284,284,tura,25.5198,90.2201,24.31,65,9,1.76,IN,05/07/2024
293,293,kill devil hills,36.0307,-75.6760,26.00,76,0,3.09,US,05/07/2024
302,302,baniachang,24.5167,91.3667,27.64,61,0,0.64,BD,05/07/2024
404,404,south hedland,-20.4089,118.5983,22.96,78,0,2.06,AU,05/07/2024


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
35,port alfred,ZA,-33.5906,26.8910,59,
107,brisas de zicatela,MX,15.8369,-97.0419,74,
171,santa terezinha,BR,-29.9002,-50.0904,65,
177,cidreira,BR,-30.1811,-50.2056,63,
236,teknaf,BD,20.8624,92.3058,74,
263,prado,BR,-17.3411,-39.2208,67,
284,tura,IN,25.5198,90.2201,65,
293,kill devil hills,US,36.0307,-75.6760,76,
302,baniachang,BD,24.5167,91.3667,61,
404,south hedland,AU,-20.4089,118.5983,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
params = {
    "categories":"accommodation.hotel",
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request in JSON format using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port alfred - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: Casa de Olas
santa terezinha - nearest hotel: Pousada Santa Terezinha
cidreira - nearest hotel: Hotel Castelo
teknaf - nearest hotel: No hotel found
prado - nearest hotel: Pousada Recanto do Prado
tura - nearest hotel: Hotel Polo Orchid
kill devil hills - nearest hotel: Mariner Days Inn & Suites
baniachang - nearest hotel: No hotel found
south hedland - nearest hotel: Port Haven
grand baie - nearest hotel: Safari
copala - nearest hotel: No hotel found
horsham - nearest hotel: Premier Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
35,port alfred,ZA,-33.5906,26.8910,59,No hotel found
107,brisas de zicatela,MX,15.8369,-97.0419,74,Casa de Olas
171,santa terezinha,BR,-29.9002,-50.0904,65,Pousada Santa Terezinha
177,cidreira,BR,-30.1811,-50.2056,63,Hotel Castelo
236,teknaf,BD,20.8624,92.3058,74,No hotel found
263,prado,BR,-17.3411,-39.2208,67,Pousada Recanto do Prado
284,tura,IN,25.5198,90.2201,65,Hotel Polo Orchid
293,kill devil hills,US,36.0307,-75.6760,76,Mariner Days Inn & Suites
302,baniachang,BD,24.5167,91.3667,61,No hotel found
404,south hedland,AU,-20.4089,118.5983,78,Port Haven


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "EsriWorldStreetMap", 
    size = "Humidity", 
    scale = 0.7,
    color = "City",
    hover_cols = ["Lng", "Lat", "City", "Humidity", "Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)